<a href="https://colab.research.google.com/github/Aerobrid/RAG_Notes_Chatbot/blob/main/RAG_workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers langchain

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd

with open("train.txt", "r") as f:
  data = f.read()

# First chunk
data[:1000]

'4\tCatheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries. The groups of patients were similar for age (left anterior descending coronary artery, 59 years; right coronary artery, 58 years; circumflex coronary artery, 62 years), patients with multivessel disease (left anterior descending coronary artery, 55%; right coronary artery, 55%; circumflex coronary artery, 64%), and patients with initial grade 0/1 antegrade flow (left anterior descending coronary artery, 79%; right coronary artery, 84%; circumflex coronary artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left anterior des

In [ ]:
# Second chunk
data[1000:2000]

'nding coronary artery, four with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery. Major catheterization laboratory events (cardioversion, cardiopulmonary resuscitation, dopamine or intra-aortic balloon pump support for hypotension, and urgent surgery) occurred in 10 patients with infarction of the left anterior descending coronary artery, eight with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery (16 of 16 shock and six of 234 nonshock patients, p less than 0.001). There was one in-laboratory death (shock patient with infarction of the left anterior descending coronary artery). \n5\tRenal abscess in children. Three cases of renal abscesses in children are described to illustrate the variable presenting features. An additional 23 pediatric cases, reported over the past ten years, were reviewed for clinical features and therapy. Fever, loin pain, and leukocytosis were common presentin

In [ ]:
from langchain.docstore.document import Document as LangChainDocument

# put text in langchain object (a document)
raw_database = LangChainDocument(page_content=data)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Better way to chunk data (split it) into smaller segments
# Reduces the computation needed for reading large files with a lot of text
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    length_function=len,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
    )

# Print 1 chunk of data
processed_data = splitter.split_documents([raw_database])
processed_data[1]

Document(metadata={}, page_content='artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left anterior descending coronary artery, four with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery. Major catheterization laboratory events (cardioversion, cardiopulmonary resuscitation, dopamine or intra-aortic balloon pump support for hypotension, and urgent surgery) occurred in 10 patients with infarction of the left anterior descending coronary artery, eight with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery (16 of 16 shock and six of 234 nonshock patients, p less than 0.001). There was one in-laboratory death (shock patient with infarction of the left anterior descending coronary artery).')

In [ ]:
processed_data[1].page_content # Just shows the raw text

'artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left anterior descending coronary artery, four with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery. Major catheterization laboratory events (cardioversion, cardiopulmonary resuscitation, dopamine or intra-aortic balloon pump support for hypotension, and urgent surgery) occurred in 10 patients with infarction of the left anterior descending coronary artery, eight with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery (16 of 16 shock and six of 234 nonshock patients, p less than 0.001). There was one in-laboratory death (shock patient with infarction of the left anterior descending coronary artery).'

In [ ]:
!pip install langchain_community sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Embedding model we are using
model_name = "thenlper/gte-small"     # username/model

In [ ]:
# Takes text and converts it into embeddings (vector representation)
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    multi_process=True,
    model_kwargs={"device": "cpu"},              # Use the gpu (cuda) instead of the cpu
    encode_kwargs={"normalize_embeddings": True}  # Set true for cosine similarity
)


<ipython-input-10-24517bafcd2d>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embedding_model.embed_query("Hello I am testing this model! :)")  # Converts text from query to vectors/embeddings

[-0.07261472940444946,
 -0.021638894453644753,
 0.059445276856422424,
 0.005138556007295847,
 0.036160048097372055,
 0.022821983322501183,
 0.06328646838665009,
 0.06366141140460968,
 0.03274865448474884,
 0.008811024017632008,
 0.01457231119275093,
 -0.09227607399225235,
 0.03334394469857216,
 0.012473110109567642,
 0.014942745678126812,
 -0.028983473777770996,
 0.03927834331989288,
 -0.03150731697678566,
 -0.03336869552731514,
 0.04447465389966965,
 0.02336178533732891,
 -0.012332630343735218,
 -0.033820074051618576,
 -0.02395055815577507,
 0.02428525499999523,
 0.02648078463971615,
 0.017138618975877762,
 -0.017410341650247574,
 -0.00035199333797208965,
 -0.2696845829486847,
 -0.026909222826361656,
 -0.042522184550762177,
 0.059325169771909714,
 -0.030579928308725357,
 -0.010632079094648361,
 -0.06484951823949814,
 -0.029427817091345787,
 0.01454528421163559,
 -0.06857765465974808,
 0.011049801483750343,
 0.05095573514699936,
 0.01972174271941185,
 0.01936781220138073,
 -0.009550146

In [ ]:
len(embedding_model.embed_query("Hello I am testing this model! :)"))

384

In [ ]:
!pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.2 MB/s eta 0:00:00


In [ ]:
from pinecone import Pinecone

# Connect to pinecone index (vector database)
pc = Pinecone(api_key="Your_api_key")
index = pc.Index("lab-rag-index")

In [ ]:
# push dataset into vector database
data_to_add = []

for i, entry in tqdm(enumerate(processed_data[:5])):      # First 5 chunks from processed_data
  text = entry.page_content                               # Getting raw text from each chunk
  vector = embedding_model.embed_query(text)              # Converting text into vectors (embedding)

  data_to_add.append({
      "id": "vec_{}".format(i),
      "values": vector,
      "metadata": {"text": text}
  })



5it [01:00, 12.06s/it]


In [ ]:
index.upsert(data_to_add, namespace="ns1")  # upsert index

{'upserted_count': 5}

In [ ]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "HuggingFaceH4/zephyr-7b-beta"

In [ ]:
!pip install -U bitsandbytes
!pip install accelerate

ERROR: Operation cancelled by user


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
llm_model = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,      # Indicates how random answer is (1: completely random | 0: Straight from external data/document, no variation whatsoever)
    max_new_tokens=500
   # top_p=0.95,
   # device_map="auto",
   # max_length=2048
)

Device set to use cuda:0


In [ ]:
llm_model("Hey whats up?")

[{'generated_text': 'Hey whats up?\n\nI’m a huge fan of your blog and I love your writing style. I’m reaching out because I’m trying to build my brand as a freelance writer and I was wondering if you’d be interested in collaborating on a guest post for my blog.\n\nMy blog focuses on [insert niche here], and I’d love to hear your thoughts on [insert topic here]. I think your insights would be valuable to my readers, and I’m confident that your writing style would resonate with them.\n\nIf you’re interested, please let me know and we can discuss further details, such as deadlines and promotional strategies.\n\nBest regards,\n\n[Your Name]\n\n[Your Email Address]\n\n[Your Phone Number]\n\n[Your Blog URL]'}]

**PROMPTING THE MODEL**

In [ ]:
prompt = """
<|system|>
You are a helpful assistant that answers on medical questions based on the real information provided from different sources and in the context.
Give the rational and well written response. If you don't have proper info in the context, answer "I don't know"
Respond only to the question asked.

<|user|>
Context:
{}
---
Here is the question you need to answer.

Question: {}
<|assistant|>
"""

In [ ]:
user_input = input("User: ")

# Creating vectors out of users question
vectorized_input = embedding_model.embed_query(user_input)

# Getting the database and retrieving most probable answer
context = index.query(
    namespace="ns1",
    vector=vectorized_input,
    top_k=1,
    include_metadata=True     # retrieve not only result(id) but text as well
)

# Call llm model using prompt and user input (the question)
answer = llm_model(prompt.format(context['matches'][0]['metadata']['text'], user_input))

print("AI response: ", answer[0]['generated_text'])

User: What is cardiogenic shock?
AI response:  
<|system|>
You are a helpful assistant that answers on medical questions based on the real information provided from different sources and in the context.
Give the rational and well written response. If you don't have proper info in the context, answer "I don't know"
Respond only to the question asked.

<|user|>
Context:
artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left anterior descending coronary artery, four with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery. Major catheterization laboratory events (cardioversion, cardiopulmonary resuscitation, dopamine or intra-aortic balloon pump support for hypotension, and urgent surgery) occurred in 10 patients with infarction of the left anterior descending coronary artery, eight with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery (16 of 16 shock and s

In [ ]:
context['matches'][0]['metadata']['text']

'artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left anterior descending coronary artery, four with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery. Major catheterization laboratory events (cardioversion, cardiopulmonary resuscitation, dopamine or intra-aortic balloon pump support for hypotension, and urgent surgery) occurred in 10 patients with infarction of the left anterior descending coronary artery, eight with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery (16 of 16 shock and six of 234 nonshock patients, p less than 0.001). There was one in-laboratory death (shock patient with infarction of the left anterior descending coronary artery).'